In [3]:
# This code was designed to find the best materials for your envelope based on the overal R value of the envelope (based on personal library) and the cost of the materials.
# You'll need to first to modify the files list_surfaces.json with areas of the surfaces of the envelope, and the material_library.json with the materials and their properties.

# Extracting surfaces area

import json
from pathlib import Path
import random


# Define the path to the JSON file relative to the current working directory
file_path = Path.cwd() / 'list_surfaces.json'

# Read json file
with open(file_path, 'r') as file:
    data = json.load(file)


# Get the areas
surface_areas = {}
for name, info in data.items():
    if isinstance(info, dict) and 'area' in info:
        surface_areas[name] = info['area']


print (surface_areas)


{'Wall1': 100.0, 'Wall2': 112.0, 'Wall3': 100.0, 'Wall4': 120.0, 'Floor1': 100.0, 'Floor2': 100.0, 'Ceiling1': 100.0, 'Ceiling2': 100.0, 'Roof': 200.0, 'Door1': 15.0, 'Door2': 15.0, 'Window1': 20.0, 'Window2': 20.0}


In [4]:
# Assign materials to  layers 
import json

# Define the path to the JSON file relative to the current working directory
json_file_path = Path.cwd() / 'material_library.json'

with open(json_file_path, 'r') as json_file:
    material_library = json.load(json_file)

# Function to get materials by category from the loaded JSON library
def get_materials_by_category(category_name):
    return [material for material in material_library if material['category'] == category_name]

# Create material categories from the JSON library
# You can choose to assign the material by category name or by the material name

# Example: Assign materials by category
floor = get_materials_by_category("floor")
ceiling = get_materials_by_category("ceiling")
roof = get_materials_by_category("roof")
wall = get_materials_by_category("wall")
window = get_materials_by_category("window")
door = get_materials_by_category("door")

# Example: Assign materials by material name
# floor = [material for material in material_library if material['name'] == "Hardwood + Concrete Slab + Plaster"]


# Update the layer_material_categories with the new material lists
layer_material_categories = {
    'Wall1': wall,
    'Wall2': wall,
    'Wall3': wall,
    'Wall4': wall,
    'Floor1': floor,
    'Floor2': floor,
    'Ceiling1': floor,
    'Ceiling2': floor,
    'Roof': roof,
    'Door1': door,
    'Door2': door,
    'Window1': window,
    'Window2': window,

}

print(layer_material_categories)



{'Wall1': [{'name': 'Fiberglass Insulation + Gypsum Board + Concrete Block', 'category': 'wall', 'R_value': 5.35, 'cost_per_sqm': 45}, {'name': 'Concrete Block + Fiberglass Insulation + Foam Board Insulation', 'category': 'wall', 'R_value': 9.9, 'cost_per_sqm': 80}, {'name': 'Fiberglass Insulation + Plaster + Gypsum Board', 'category': 'wall', 'R_value': 4.47, 'cost_per_sqm': 60}, {'name': 'Fiberglass Insulation + Gypsum Board + Foam Board Insulation', 'category': 'wall', 'R_value': 8.15, 'cost_per_sqm': 75}, {'name': 'Brick + Fiberglass Insulation + Drywall', 'category': 'wall', 'R_value': 6.0, 'cost_per_sqm': 22}, {'name': 'Concrete Block + Spray Foam Insulation + Gypsum Board', 'category': 'wall', 'R_value': 10.0, 'cost_per_sqm': 28}, {'name': 'Metal Stud + Fiberglass Batting + Gypsum Board', 'category': 'wall', 'R_value': 8.2, 'cost_per_sqm': 25}, {'name': 'Wood Stud + Cellulose Insulation + Plaster', 'category': 'wall', 'R_value': 7.5, 'cost_per_sqm': 20}, {'name': 'Wood Stud + Fi

In [5]:
def calculate_totalR(solution):
    
    total_R = 0

    total_area = sum(surface_areas.values())

    for layer, material_index in zip(layer_material_categories.keys(), solution):
        # Retrieve the material using its index from the layer in layer_material_categories
        material = layer_material_categories[layer][material_index]

        # Retrieve the R-value for the selected material
        material_R = material['R_value']

        # Retrieve the surface area for the layer
        area = float(surface_areas[layer])  # Convert the area to a list or iterable object

 
        # Calculate the total R-value for all the layers
        total_R += area * material_R
    
    overall_R = total_R / total_area

    return overall_R

calculate_totalR([1, 3, 3, 2, 0, 0,1, 3, 3, 2, 0, 0,5])


6.017876588021779

In [6]:
def calculate_totalC(solution):
    
    total_C = 0

    for layer, material_index in zip(layer_material_categories.keys(), solution):

        
        # Retrieve the material using its index from the layer in layer_material_categories
        material = layer_material_categories[layer][material_index]

        # Retrieve the R-value for the selected material
        material_C = material['cost_per_sqm']

        # Retrieve the surface area for the layer
        area = float(surface_areas[layer])  # Convert the area to a list or iterable object

        # Calculate the total R-value for all the layers
        total_C += material_C * area

    return total_C 

calculate_totalC([1, 3, 3, 2, 0, 0,1, 3, 3, 2, 0, 0,5])


87350.0

In [7]:
import numpy as np

def fitness_function(solution, weight_R, weight_C, target_C=None):
    total_R = calculate_totalR(solution)
    total_C = calculate_totalC(solution)

    # Normalize R and C values
    max_R = max(layer['R_value'] for layers in layer_material_categories.values() for layer in layers) * sum(surface_areas.values())
    max_C = max(layer['cost_per_sqm'] for layers in layer_material_categories.values() for layer in layers) * sum(surface_areas.values())


    norm_R = total_R / max_R


    # Calculate initial fitness based on total_R and weight_R
    fitness = weight_R * norm_R

    if target_C is not None:
        # Calculate deviations from the target cost
        deviations = np.abs(total_C - target_C)
     
        # Normalize deviation
        norm_C_d = deviations / max_C

        # Apply penalties to the fitness score
        fitness -=  norm_C_d  * weight_C
    else:
        # If no target cost is provided, just subtract the total cost scaled by weight_C
        fitness -= total_C * weight_C

    return fitness

#print(fitness_function([1, 3, 3, 2, 0, 0, 1, 3, 3, 2, 0, 0, 5], weight_R=0.8, weight_C=0.2, target_C=60000))




-0.019417939993610037


In [8]:
#Checking the number of material per categories

solutions = []


# The length of materials for each layer
len_mat = [len(materials) for materials in layer_material_categories.values()]
print(len_mat)

[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10]


In [13]:
# Initialize the number of materials for each layer
len_mat = [len(layer_material_categories[layer]) for layer in layer_material_categories.keys()]

# Generate initial solutions
solutions = [
    tuple(random.randint(0, length - 1) for length in len_mat)
    for _ in range(500000)  
]

best_overall_solution = (-float('inf'), None)  # Initialize with worst possible score and no solution

# Iterate through generations
for i in range(1000):
    ranked_solutions = []
    for s in solutions:


        # Please adjust the weights and target cost as needed
        
        score = fitness_function(s, weight_R=0.95, weight_C=0.05, target_C=60000)

        ranked_solutions.append((score, s))
    
    ranked_solutions.sort(reverse=True)

    print(f"=== Gen {i} best solution ===")
    print(ranked_solutions[0])

    best_solutions = ranked_solutions[:100]

    if ranked_solutions[0][0] > best_overall_solution[0]:
        best_overall_solution = ranked_solutions[0]

    num_gene_positions = len(best_solutions[0][1])
    element_groups = [[] for _ in range(num_gene_positions)]
    for _, solution in best_solutions:
        for pos in range(num_gene_positions):
            element_groups[pos].append(solution[pos])

    new_gen = []
    for _ in range(1000):
        new_solution = [random.choice(elements) for elements in element_groups]
        if random.random() < 0.1:
            new_solution[random.randint(0, num_gene_positions - 1)] = random.randint(0, len_mat[random.randint(0, num_gene_positions - 1)] - 1)
        new_gen.append(tuple(new_solution))

    solutions = new_gen

print("=== Best Overall Solution ===")
print(best_overall_solution)
    


=== Gen 0 best solution ===
(0.0006693879466800175, (9, 9, 9, 1, 1, 1, 3, 8, 8, 2, 8, 3, 2))
=== Gen 1 best solution ===
(0.0006657550535077289, (1, 5, 6, 5, 3, 1, 3, 3, 8, 7, 2, 4, 0))
=== Gen 2 best solution ===
(0.0006883503348144439, (3, 9, 5, 9, 1, 1, 1, 1, 6, 5, 2, 6, 6))
=== Gen 3 best solution ===
(0.0007023984604793791, (5, 6, 1, 5, 3, 3, 3, 1, 5, 0, 7, 1, 8))
=== Gen 4 best solution ===
(0.0006749264659866073, (6, 5, 3, 5, 3, 6, 7, 3, 2, 7, 9, 5, 9))
=== Gen 5 best solution ===
(0.0006642155328869141, (5, 6, 5, 5, 3, 9, 0, 3, 2, 7, 5, 5, 8))
=== Gen 6 best solution ===
(0.0006910570123286813, (5, 5, 5, 9, 3, 3, 6, 1, 1, 7, 7, 6, 9))
=== Gen 7 best solution ===
(0.0006705206833969586, (5, 6, 7, 5, 3, 9, 3, 3, 2, 7, 7, 4, 6))
=== Gen 8 best solution ===
(0.0006840290381125225, (7, 5, 5, 9, 3, 3, 6, 3, 2, 5, 7, 6, 6))
=== Gen 9 best solution ===
(0.0006801489454909569, (5, 5, 5, 5, 3, 6, 0, 3, 2, 4, 7, 4, 4))
=== Gen 10 best solution ===
(0.0007065304462106514, (7, 5, 5, 5, 3, 3

In [14]:
#Print the results!

# Get the ordered list of layer keys from 'layer_material_categories'
layer_keys_ordered = list(layer_material_categories.keys())

# Mapping layers to their respective material indices based on the best solution
best_solution_materials = {
    layer: layer_material_categories[layer][index]['name']
    for layer, index in zip(layer_keys_ordered, best_overall_solution[1])
}

# Calculate the total cost and total R-value of the best solution
total_price = calculate_totalC(best_overall_solution[1])
total_R = calculate_totalR(best_overall_solution[1])

# Print the materials for each layer in a formatted way
print(f"Best Solution with Fitness Score: {best_overall_solution[0]}\n")

print("Selected materials for each layer:")
for layer, material_name in best_solution_materials.items():
    print(f"{layer.capitalize()}: {material_name}")

print("Total price:", total_price)
print("R total:", total_R)


Best Solution with Fitness Score: 0.000706843356905939

Selected materials for each layer:
Wall1: Concrete Block + Spray Foam Insulation + Gypsum Board
Wall2: Concrete Block + Spray Foam Insulation + Gypsum Board
Wall3: Wood Stud + Cellulose Insulation + Plaster
Wall4: Concrete Block + Spray Foam Insulation + Gypsum Board
Floor1: Carpet + Foam Board Insulation + Gypsum Board
Floor2: Carpet + Foam Board Insulation + Gypsum Board
Ceiling1: Laminate Flooring + Cork Underlayment + Concrete Slab
Ceiling2: Carpet + Foam Board Insulation + Gypsum Board
Roof: Clay Tile + Foam Board Insulation + Fiberglass Insulation
Door1: Glass Door + Polyurethane Foam + Gypsum Board
Door2: Glass Door + Polyurethane Foam + Gypsum Board
Window1: Triple-pane Window + Krypton Gas + Double Low-E Glass
Window2: Triple-pane Window + Argon Gas + Double Low-E Glass
Total price: 59996.0
R total: 8.207803992740471
